# Dataset D1-D4 generation:

- Direct mapping + scene + target object
- 11 Actions + 9 Gestures
- 10000 draws
- CM feature templates:
    - `D0` 2D `[11,11]` diagonal matrix
    - `D1` 2D `[11,9]`, `11` actions, `9` gestures
    - `D2` 3D `[3,11,9]`, + `3` object types
    - `D3` 4D `[2,3,11,9]` + `2` user profiles
    - `D4` 4D `[2,2,3,11,9]` + `2` object states

- types, scene and other features are not used
- scene: n objects - initialized randomly
- target object generation type: **random**

#### Different dataset variations:

- `D0` to `D3`, uses `CM0` to `CM3`
    - `D0` - gestures (diagonal 2D matrix `11 x 11`) (one gesture per action)
    - `D1` - gestures (more gestures to one actions)
    - `D2` - gestures + object type 
    - `D3` - gestures + object type + user
- `D0_2gs` to `D3_2gs`, uses `CM0 to CM3`, user perform two gestures in one episode
    - `D0_2gs` - similar as `D0`
    - `D1_2gs` - similar as `D1`
    - `D2_2gs` - similar as `D2`
    - `D3_2gs` - similar as `D3`
- `D1_3obj` to `D3_3obj`, uses `CM1` to `CM3`, const. 3 objects on the scene
    - `D1_3obj` - similar as `D1`
    - `D2_3obj` - similar as `D2`
    - `D3_3obj` - similar as `D3`
- `D0_1` to `D4_1`, uses `CM0`, `CM1_1` to `C4_1`. Final versions, uses single gesture per action for given context
    - `D1_1` - gestures
    - `D2_1` - gestures + object types
    - `D3_1` - gestures + object types + user profile
    - `D4_1` - gestures + object types + user profile + object state
    
Final versions D1_1 to C4_1 are the final version of datasets D1, D2, D3, D4

TODO:
- [ ] Code clean-up
- [ ] Put CM values to more suitable import form

In [1]:
from srcmodules import Robots, Tasks, Users
from srcmodules.Objects import Object
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
from srcmodules.utils import show_each_action_occurance
import numpy as np

from numpy import array as a
import os

In [2]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'five', 'grab', 'thumbsup', 'rotate', 'point', 'two', 'three']
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [3]:
CM0 = np.zeros((lenA,lenG)) # intents x gestures
# gestures:           up,left,dwn,rght,fiv,grab,thum,rota,poin,two,thre
CM0[:,:] = np.array([[ 1,  .0, .0,  .0, .0, .0,   .0,  .0,  .0, .0, .0], # move_up
                     [.0,   1, .0,  .0, .0, .0,   .0,  .0,  .0, .0, .0], # move_left
                     [.0,  .0,  1,  .0, .0, .0,   .0,  .0,  .0, .0, .0], # move_down
                     [.0,  .0, .0,   1, .0, .0,   .0,  .0,  .0, .0, .0], # move_right
                     [.0,  .0, .0,  .0,  1, .0,   .0,  .0,  .0, .0, .0], # put
                     [.0,  .0, .0,  .0, .0,  1,   .0,  .0,  .0, .0, .0], # put_on
                     [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0, .0, .0], # pour
                     [.0,  .0, .0,  .0, .0, .0,   .0,   1,  .0, .0, .0], # pick_up
                     [.0,  .0, .0,  .0, .0, .0,   .0,  .0,   1, .0, .0], # place
                     [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0,  1, .0], # open
                     [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0, .0,  1]])# close

In [4]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'five', 'grab', 'thumbsup', 'rotate', 'point']
Object.all_types = Otypes = ['cup', 'drawer', 'object']
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [5]:
CM1_1 = np.zeros((lenA,lenG)) # object types x intents x gestures
# gestures:            up,left,dwn,rght,fiv,grab,thum,rota,poin
CM1_1[:,:] =np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                      [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                      [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                      [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                      [.0,  .0, .0,  .0,  1, .0,   .0,  .0,  .0], # put
                      [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                      [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                      [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                      [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                      [.0,  .0, .0,  .0, .0, .0,   .0,  .0,   1], # open
                      [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [6]:
CM2_1 = np.zeros((3, lenA,lenG)) # object types x intents x gestures
# objtyp 0: cup
# gestures:             up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2_1[0,:,:] =np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 1: drawer
# gestures:             up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2_1[1,:,:] =np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0,  0,  .0,  1, .0,   .0,  .0,  .0], # put
                       [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                       [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
# gestures:             up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2_1[2,:,:] =np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [7]:
CM3_1 = np.zeros((2,3, lenA,lenG)) # users x object types x intents x gestures
# --- USER 1 ---
# objtyp 0: cup
# gestures:                up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[0,0,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                          [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                          [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                          [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                          [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                          [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                          [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
# gestures:                up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[0,1,:,:] = np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                          [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                          [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                          [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                          [.0,  .0,  0,  .0,  1, .0,   .0,  .0,  .0], # put
                          [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                          [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                          [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                          [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
# gestures:                up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[0,2,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                          [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                          [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                          [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                          [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                          [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                          [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                          [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                          [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

# --- USER 2 ---
# objtyp 0: cup
# gestures:                up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[1,0,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                          [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                          [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                          [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                          [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                          [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                          [.0,  .0, .0,  .0,  1, .0,    0,  .0,  .0], # place
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
# gestures:                up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[1,1,:,:] = np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                          [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                          [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                          [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                          [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                          [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                          [.0,  .0, .0,  .0,  1, .0,    0,  .0,   0], # place
                          [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                          [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
# gestures:                up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[1,2,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                          [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                          [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                          [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                          [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                          [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                          [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                          [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                          [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                          [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [5]:
CM4_1 = np.zeros((2,2,3, lenA,lenG)) # object_state x users x object types x intents x gestures
################ ================ STATE 0 ====================
# --- USER 0 ---
# user 0, objtyp 0: cup : EMPTY
# gestures:                     up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,0,:,:] =  np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer : CLOSED
# gestures:                     up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,1,:,:] =  np.array([[ 0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0,  0,  .0,  0, .0,   .0,  .0,  .0], # put
                               [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                               [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,    0,  .0,  .0]])# close
# user 0, objtyp 2: object/box : NOT BLOCKED
# gestures:                     up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,2,:,:] =   np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# --- USER 1 ---
# user 1, objtyp 0: cup : EMPTY
# gestures:                   up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,0,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                             [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                             [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                             [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                             [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                             [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                             [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                             [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                             [.0,  .0, .0,  .0,  1, .0,    0,  .0,  .0], # place
                             [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                             [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 1: drawer : CLOSED
# gestures:                 up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,1,:,:]=np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   0], # put
                           [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0,  1, .0,    0,  .0,   0], # place
                           [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# user 1, objtyp 2: object/box : NOT BLOCKED
# gestures:                 up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,2,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                           [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                           [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

################ ================ STATE 1 ====================

# --- USER 0 ---
# user 0, objtyp 0: cup : FULL
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[1, 0,0,:,:] =  np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer : OPENED
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[1, 0,1,:,:] =  np.array([[ 0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0,  0,  .0,  1, .0,   .0,  .0,  .0], # put
                               [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                               [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# user 0, objtyp 2: object/box : BLOCKED
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[1, 0,2,:,:] =   np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  0,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# --- USER 1 ---
# user 1, objtyp 0: cup : FULL
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[1, 1,0,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                           [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0,  1, .0,    0,  .0,  .0], # place
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 1: drawer : OPENED
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[1, 1,1,:,:]=np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                           [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0,  1, .0,    0,  .0,   0], # place
                           [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# user 1, objtyp 2: object/box : BLOCKED
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[1, 1,2,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                           [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                           [ 0,  .0, .0,  .0, .0,  0,    0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

### The more gestures for one actions CMs
- Ambiguous form in this setup

In [9]:
CM1 = np.zeros((lenA,lenG)) # actions x gestures
                    # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM1[:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                     [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                     [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                     [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                     [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                     [.0,  .0,  1,  .0, .0, .0,   .0,  .0,  .0], # put_on
                     [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                     [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                     [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                     [.5,  .0, .0,  .0, .0, .5,   .0,  .0,  .0], # open
                     [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close

In [10]:
CM2 = np.zeros((3, lenA,lenG)) # object types x actions x gestures
# objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                       [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [11]:
CM3 = np.zeros((2,3, lenA,lenG)) # users x object types x actions x gestures
CM3[0,0,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,1,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                         [.0,  .0,  0,  .0, .0, .0,    1,  .0,   0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,   0], # place
                         [ 0,  .0, .0,  .0, .0,  0,    0,   1,  .0], # open
                         [.0,  .0,  0,  .0,  1,  0,   .0,  .0,  .0]])# close
# user 0, objtyp 2: object = box
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,2,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  1, .0,   .0,  .0,   0], # put
                         [.0,  .0,  0,  .0, .0, .0,   .0,  .0,   1], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,   0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[1,0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 1: drawer
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[1,1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                         [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# user 1, objtyp 2: object = box
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[1,2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

helper functions

In [13]:
def get_CM_table_from_dataset_name(name):
    ''' Possibility to add dataset variations
    E.g. dataset name: D3_1_variation9 -> CM table: CM3_1
    '''
    if   'D0_1' in name:
        CM = CM0_1
    elif 'D1_1' in name:
        CM = CM1_1
    elif 'D2_1' in name:
        CM = CM2_1
    elif 'D3_1' in name:
        CM = CM3_1
    elif 'D4_1' in name:
        CM = CM4_1
    elif 'D0' in name:
        CM = CM0
    elif 'D1' in name:
        CM = CM1
    elif 'D2' in name:
        CM = CM2
    elif 'D3' in name:
        CM = CM3
    elif 'D5' in name:
        CM = CM4_1
    else: raise Exception("not the right name of dataset")
    return CM

### Configuration

- Edit here!
- options: `dataset_name = D1_1`, `dataset_name = D2_1`, `dataset_name = D3_1`, `dataset_name = D4_1` 
`dataset_name=D5`


In [22]:
draws = 10000
dataset_name = 'D5'
n_of_performed_gestures = 1
# experimental
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [23]:
def create_observation(s,TaTo, dataset_name, n_of_performed_gestures=1):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    user_id = s.u.selected_id
    object_state = s.objects[target_object_id].get_unique_state()

    # Pick gestures for dataset based on scene context and action id from CM table
    if   'D0_1' in dataset_name:
        gesture_probs_intent = CM0_1[target_action_id,:]
    elif 'D1_1' in dataset_name:
        gesture_probs_intent = CM1_1[target_action_id,:]
    elif 'D2_1' in dataset_name:
        gesture_probs_intent = CM2_1[target_object_type_id,target_action_id,:]
    elif 'D3_1' in dataset_name:
        gesture_probs_intent = CM3_1[user_id,target_object_type_id,target_action_id,:]
    elif 'D4_1' in dataset_name:
        gesture_probs_intent = CM4_1[object_state,user_id,target_object_type_id,target_action_id,:]
    elif 'D1' in dataset_name:
        gesture_probs_intent = CM1[target_action_id,:]
    elif 'D2' in dataset_name:
        gesture_probs_intent = CM2[target_object_type_id,target_action_id,:]
    elif 'D3' in dataset_name:
        gesture_probs_intent = CM3[user_id,target_object_type_id,target_action_id,:]
    elif 'D0' in dataset_name:
        gesture_probs_intent = CM0[target_action_id,:]
    elif 'D5' in dataset_name:
        gesture_probs_intent = CM4_1[object_state,user_id,target_object_type_id,target_action_id,:]
    else: raise Exception("Wrong name")
    if sum(gesture_probs_intent) == 0: return False, False
    
    try:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=False)
    except ValueError:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=True)

        
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    
    focus_point = abs(np.random.normal(loc = s.objects[target_object_id].position, scale = 0.1,size =(1,3)))
    
    return gesture_vec, focus_point

In [24]:
dataset = []
# Generate random action
while len(dataset) < draws:

    n_objects = round(np.random.normal(3.5,1))
    if n_objects > 7: n_objects = 7
    if n_objects < 2: n_objects = 2
    init = ",".join(object_types_order_list[0:n_objects])
    
    s = Scene(init=init, random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    if TaTo is False: continue
    
    gobs, oobs = create_observation(s, TaTo, dataset_name, n_of_performed_gestures=n_of_performed_gestures)
    if gobs is False: continue
    
    dataset.append((s.to_dict(), TaTo, gobs, oobs))
    
    if len(dataset) % 2000 == 0: print(f"{len(dataset)} samples generated!")

2000 samples generated!
4000 samples generated!
6000 samples generated!
8000 samples generated!
10000 samples generated!


In [16]:
def save_with_config(dataset, name):
    config = {
        'A': A,
        'G': G,
        'Otypes': Otypes,
        'lenA': lenA,
        'lenG': lenG,
        'lenOtypes': lenOtypes,
        'CM': get_CM_table_from_dataset_name(name),
    }
    np.save(f"{os.path.abspath('./datasets')}/{name}", np.asanyarray({'dataset': dataset, 'config': config}, dtype=object))

In [25]:
save_with_config(dataset, name=dataset_name)

Futher tests
- Optional -> only to get info abou generated dataset

In [21]:
show_each_action_occurance(dataset, A)

Action occurances: [ 870. 1553. 2115. 1540.  286.  178.   38. 1167. 1134.  269.  850.]


Some actions were not feasible and were not triggered as often, this can be a problem, when evaluating dataset.
- D0 action occurances: [  x 1869. 2474. 1879.  489.  247.   23. 1032. 1077.  249.  661.]
- D1 action occurances: [  x 1838. 2564. 1848.  473.  276.   32. 1061. 1040.  214.  644.]
- D2 action occurances: [  x 1854. 2504. 1913.  507.  253.   23. 1047. 1009.  221.  653.]
- D3 action occurances: [  x 1850. 2566. 1854.  489.  257.   31. 1029. 1010.  248.  666.]


In [1]:
A = [870, 1869., 2474., 1879.,  489.,  247.,   23., 1032., 1077.,  249.,  661.]

In [2]:
sum(A)

10870.0

In [3]:
240/10870

0.02207911683532659

# Dataset D5 generation

- Same as `D4_1` + only added object shuffling
- Uses `CM4_1` matrix defined earlier!

In [1]:
from srcmodules import Robots, Tasks, Users
from srcmodules.Objects import Object
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
import numpy as np

from numpy import array as a
import os
from copy import deepcopy

In [2]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'five', 'grab', 'thumbsup', 'rotate', 'point']
Object.all_types = Otypes = ['cup', 'drawer', 'object']
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [7]:
draws = 10000
dataset_name = 'D5'
n_of_performed_gestures = 1
# experimental
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [8]:
def create_observation(s,TaTo, dataset_name, n_of_performed_gestures=1):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    user_id = s.u.selected_id
    object_state = s.objects[target_object_id].get_unique_state()
    
    if 'D5' in dataset_name:
        gesture_probs_intent = CM4_1[object_state,user_id,target_object_type_id,target_action_id,:]
    else: raise Exception("Wrong name")
    if sum(gesture_probs_intent) == 0: return False, False
    
    try:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=False)
    except ValueError:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=True)

        
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    
    focus_point = abs(np.random.normal(loc = s.objects[target_object_id].position, scale = 0.1,size =(1,3)))
    
    return gesture_vec, focus_point

In [10]:
dataset = []
# Generate random action
while len(dataset) < draws:

    n_objects = round(np.random.normal(3.5,1))
    if n_objects > 7: n_objects = 7
    if n_objects < 2: n_objects = 2
    objlistshuffled = deepcopy(object_types_order_list)
    np.random.shuffle(object_types_order_list)
    init = ",".join(objlistshuffled[0:n_objects])
    
    s = Scene(init=init, random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    if TaTo is False: continue
    
    gobs, oobs = create_observation(s, TaTo, dataset_name, n_of_performed_gestures=n_of_performed_gestures)
    if gobs is False: continue
    
    dataset.append((s.to_dict(), TaTo, gobs, oobs))
    
    if len(dataset) % 2000 == 0: print(f"{len(dataset)} samples generated!")

2000 samples generated!
4000 samples generated!
6000 samples generated!
8000 samples generated!
10000 samples generated!


In [11]:
def save_with_config(dataset, name):
    config = {
        'A': A,
        'G': G,
        'Otypes': Otypes,
        'lenA': lenA,
        'lenG': lenG,
        'lenOtypes': lenOtypes,
        'CM': CM4_1,
    }
    np.save(f"{os.path.abspath('./datasets')}/{name}", np.asanyarray({'dataset': dataset, 'config': config}, dtype=object))

In [12]:
save_with_config(dataset, name=dataset_name)

# Dataset D6

- Different setup, updated set of gestures

In [1]:
from srcmodules import Robots, Tasks, Users
from srcmodules.Objects import Object
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
import numpy as np

from numpy import array as a
import os
from copy import deepcopy

In [2]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'pick_up', 'put', 'place', 'pour', 'push', 'replace']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'grab', 'point', 'two', 'three', 'four', 'five', 'thumbsup']
Object.all_types = Otypes = ['cup', 'drawer', 'object']
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [3]:
draws = 10000
dataset_name = 'D6'
n_of_performed_gestures = 1
# experimental
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [4]:
def create_observation(s,TaTo, dataset_name, n_of_performed_gestures=1):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    user_id = s.u.selected_id
    object_state = s.objects[target_object_id].get_unique_state()
    
    if 'D6' in dataset_name:
        gesture_probs_intent = CM6[object_state,user_id,target_object_type_id,target_action_id,:]
    else: raise Exception("Wrong name")
    if sum(gesture_probs_intent) == 0: return False, False
    
    try:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=False)
    except ValueError:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=True)

        
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    
    focus_point = abs(np.random.normal(loc = s.objects[target_object_id].position, scale = 0.1,size =(1,3)))
    
    return gesture_vec, focus_point

In [5]:
CM6 = np.zeros((2,2,3, lenA,lenG)) # object_state x users x object types x intents x gestures
################ ================ STATE 0 ====================
# --- USER 0 ---
# user 0, objtyp 0: cup : EMPTY
# gestures:                  up,left,down,rght,grab,point,two,thre,four,five,thumbsup
CM6[0, 0,2,:,:] = np.array([[ 1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_up
                            [ 0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_left
                            [ 0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0], # move_down
                            [ 0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0], # move_right
                            [ 0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0], # pick_up
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1], # put
                            [ 0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0], # place
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0], # pour
                            [ 0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0], # push
                            [ 0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0]])# replace
# user 0, objtyp 1: drawer : CLOSED
# gestures:                     up,left,down,rght,grab,point,two,thre,four,five,thumbsup
CM6[0, 0,1,:,:] =  np.array([[ 1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_up
                            [ 0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_left
                            [ 0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0], # move_down
                            [ 0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0], # move_right
                            [ 0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0], # pick_up
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1], # put
                            [ 0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0], # place
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0], # pour
                            [ 0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0], # push
                            [ 0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0]])# replace

CM6[0, 0,0,:,:] = deepcopy(CM6[0, 0,2,:,:]) # Cup is the same as object

CM6[0, 1,:,:,:] = deepcopy(CM6[0, 0,:,:,:]) # User 0,1 are the same

CM6[1, :,:,:,:] = deepcopy(CM6[0, :,:,:,:]) # States 0,1 are the same


In [6]:
dataset = []
# Generate random action
while len(dataset) < draws:

    n_objects = round(np.random.normal(3.5,1))
    if n_objects > 7: n_objects = 4
    if n_objects < 2: n_objects = 2
    objlistshuffled = deepcopy(object_types_order_list)
    np.random.shuffle(object_types_order_list)
    init = ",".join(objlistshuffled[0:n_objects])
    
    s = Scene(init=init, random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    if TaTo is False: continue
    
    gobs, oobs = create_observation(s, TaTo, dataset_name, n_of_performed_gestures=n_of_performed_gestures)
    if gobs is False: continue
    
    dataset.append((s.to_dict(), TaTo, gobs, oobs))
    
    if len(dataset) % 2000 == 0: print(f"{len(dataset)} samples generated!")

2000 samples generated!
4000 samples generated!
6000 samples generated!
8000 samples generated!
10000 samples generated!


In [7]:
dataset_name

'D6'

In [8]:
def save_with_config(dataset, name):
    config = {
        'A': A,
        'G': G,
        'Otypes': Otypes,
        'lenA': lenA,
        'lenG': lenG,
        'lenOtypes': lenOtypes,
        'CM': CM6,
    }
    np.save(f"{os.path.abspath('./datasets')}/{name}", np.asanyarray({'dataset': dataset, 'config': config}, dtype=object))
save_with_config(dataset, name=dataset_name)

In [10]:
show_each_action_occurance(dataset, A)

Action occurances: [1052. 1891. 2338. 1777.  956.  299.  915.   30.  742.    0.]


# Dataset D7

- Custom made dataset for aplication

In [10]:
from srcmodules import Robots, Tasks, Users
from srcmodules.Objects import Object
from srcmodules.Scenes import Scene
from srcmodules.Actions import Actions
from srcmodules.Gestures import Gestures
from srcmodules.Users import Users
from srcmodules.utils import show_each_action_occurance
import numpy as np

from numpy import array as a
import os
from copy import deepcopy

In [2]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'pick_up', 'put', 'place', 'pour', 'rotate', 'open', 'close']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'grab', 'pinch', 'point', 'two', 'three', 'four', 'five', 'thumbsup']
Object.all_types = Otypes = ['cup', 'drawer', 'object']
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [3]:
draws = 10000
dataset_name = 'D7'
n_of_performed_gestures = 1
# experimental
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [4]:
def create_observation(s,TaTo, dataset_name, n_of_performed_gestures=1):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    user_id = s.u.selected_id
    object_state = s.objects[target_object_id].get_unique_state()
    
    if 'D7' in dataset_name:
        gesture_probs_intent = CM6[object_state,user_id,target_object_type_id,target_action_id,:]
    else: raise Exception("Wrong name")
    if sum(gesture_probs_intent) == 0: return False, False
    
    try:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=False)
    except ValueError:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=True)

        
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    
    focus_point = abs(np.random.normal(loc = s.objects[target_object_id].position, scale = 0.1,size =(1,3)))
    
    return gesture_vec, focus_point

In [5]:
CM6 = np.zeros((2,2,3, lenA,lenG)) # object_state x users x object types x intents x gestures
################ ================ STATE 0 ====================
# --- USER 0 ---
# user 0, objtyp 0: cup : EMPTY
# gestures:                  up,left,down,rght,grab,pinc,point,two,thre,four,five,thumbsup
CM6[0, 0,2,:,:] = np.array([[ 1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_up
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_left
                            [ 0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_down
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_right
                            [ 0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0], # pick_up
                            [ 0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0], # put
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0], # place
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1], # pour
                            [ 0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0], # rotate
                            [ 0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0], # open
                            [ 0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])# close
# user 0, objtyp 1: drawer : CLOSED
# gestures:                     up,left,down,rght,grab,point,two,thre,four,five,thumbsup
CM6[0, 0,2,:,:] = np.array([[ 1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_up
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_left
                            [ 0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_down
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], # move_right
                            [ 0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0], # pick_up
                            [ 0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0], # put
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0], # place
                            [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1], # pour
                            [ 0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0], # rotate
                            [ 0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0], # open
                            [ 0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])# close

CM6[0, 0,0,:,:] = deepcopy(CM6[0, 0,2,:,:]) # Cup is the same as object

CM6[0, 1,:,:,:] = deepcopy(CM6[0, 0,:,:,:]) # User 0,1 are the same

CM6[1, :,:,:,:] = deepcopy(CM6[0, :,:,:,:]) # States 0,1 are the same


In [6]:
dataset = []
# Generate random action
while len(dataset) < draws:

    n_objects = round(np.random.normal(3.5,1))
    if n_objects > 7: n_objects = 4
    if n_objects < 2: n_objects = 2
    objlistshuffled = deepcopy(object_types_order_list)
    np.random.shuffle(object_types_order_list)
    init = ",".join(objlistshuffled[0:n_objects])
    
    s = Scene(init=init, random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    if TaTo is False: continue
    
    gobs, oobs = create_observation(s, TaTo, dataset_name, n_of_performed_gestures=n_of_performed_gestures)
    if gobs is False: continue
    
    dataset.append((s.to_dict(), TaTo, gobs, oobs))
    
    if len(dataset) % 2000 == 0: print(f"{len(dataset)} samples generated!")

2000 samples generated!
4000 samples generated!
6000 samples generated!
8000 samples generated!
10000 samples generated!


In [7]:
dataset_name

'D7'

In [8]:
def save_with_config(dataset, name):
    config = {
        'A': A,
        'G': G,
        'Otypes': Otypes,
        'lenA': lenA,
        'lenG': lenG,
        'lenOtypes': lenOtypes,
        'CM': CM6,
    }
    np.save(f"{os.path.abspath('./datasets')}/{name}", np.asanyarray({'dataset': dataset, 'config': config}, dtype=object))
save_with_config(dataset, name=dataset_name)

In [11]:
show_each_action_occurance(dataset, A)

'Action occurances: [ 939.    0. 2544.    0. 1473.  940.  909.  593. 2602.    0.    0.]'

In [12]:
A

['move_up',
 'move_left',
 'move_down',
 'move_right',
 'pick_up',
 'put',
 'place',
 'pour',
 'rotate',
 'open',
 'close']